# Maestría en Ciencia de Datos
### Autor: MSc. Jose Cardenas
### Tema: Regresión Logística aplicada a negocios

## 1.Librerías a utilizar

In [1]:
# Importar las librerías necesarias en Python
import pandas as pd 
import warnings
import re
import numpy as np
from scipy.stats import spearmanr
from sklearn.preprocessing import OneHotEncoder
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import roc_auc_score
from scipy.stats import ks_2samp
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# Conectar con Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# Funciones

# Conteo de nulos
def missing_tabla(df):
    miss_value = df.isnull().sum()
    miss_value_percent = 100*df.isnull().sum()/len(df)
    
    # Resultados
    resultado = pd.concat([miss_value,miss_value_percent], axis=1)
    resultado.rename(columns={0:'Cantidad de Missing',1:'% Porcentaje de Missing'}, inplace = True)
    resultado.sort_values(by='% Porcentaje de Missing', inplace = True, ascending = False)
    print('Cantidad de Missing por variable: ')
    return resultado

def tidy_corr_matrix(corr_mat):
    '''
    Función para convertir una matriz de correlación de pandas en formato tidy
    '''
    corr_mat = corr_mat.stack().reset_index()
    corr_mat.columns = ['variable_1','variable_2','r']
    corr_mat = corr_mat.loc[corr_mat['variable_1'] != corr_mat['variable_2'], :]
    corr_mat['abs_r'] = np.abs(corr_mat['r'])
    corr_mat = corr_mat.sort_values('abs_r', ascending=False)
    return(corr_mat)

# KS
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic

# Calcula los indicadores
def calcula_indicadores(modelo, y, y_hat):
  # AUC
  AUC = roc_auc_score(y, y_hat)

  # Gini
  GINI = 2*AUC-1

  # KS
  KS = ks_stat(y, y_hat)

  return pd.DataFrame([GINI, AUC, KS],index=['GINI','AUC', 'KS'],columns=['Indicador'])

def clean_names(model_x):
  variables =[]
  for variable in model_x.model.exog_names:
    name = variable
    if "(" in variable:
      #clean_variable = re.search("\.w+|\w+", variable)
      #print(clean_variable.string)
      name = variable[variable.find("(")+1:variable.find(")")]
    variables.append(name)
  return variables

def calcular_pesos(modelo_x):
  coef_model = pd.DataFrame(modelo_x.tvalues,columns=['t_value'])
  coef_model['t_value2'] = np.power(coef_model['t_value'],2)
  coef_model['variable'] = clean_names(modelo_x)
  coef_model['total'] = sum(coef_model['t_value2'])
  resul = coef_model.groupby('variable')['t_value2'].sum() / coef_model['total'].mean()
  resul = resul * 100
  return resul.sort_values(ascending=True)


## 2.Data a Utilizar


In [6]:
# Para importar archivos 
from google.colab import files 
files.upload()

Saving german_credit_data_test.csv to german_credit_data_test.csv
Saving german_credit_data_train.csv to german_credit_data_train.csv


{'german_credit_data_test.csv': b'Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose\r\n67,male,2,own,NA,little,1169,6,radio/TV\r\n22,female,2,own,little,moderate,5951,48,radio/TV\r\n49,male,1,own,little,NA,2096,12,education\r\n45,male,2,free,little,little,7882,42,furniture/equipment\r\n53,male,2,free,little,little,4870,24,car\r\n35,male,1,free,NA,NA,9055,36,education\r\n53,male,2,own,quite rich,NA,2835,24,furniture/equipment\r\n35,male,3,rent,little,moderate,6948,36,car\r\n61,male,1,own,rich,NA,3059,12,radio/TV\r\n28,male,3,own,little,moderate,5234,30,car\r\n25,female,2,rent,little,moderate,1295,12,car\r\n24,female,2,rent,little,little,4308,48,business\r\n22,female,2,own,little,moderate,1567,12,radio/TV\r\n60,male,1,own,little,little,1199,24,car\r\n28,female,2,rent,little,little,1403,15,car\r\n32,female,1,own,moderate,little,1282,24,radio/TV\r\n53,male,2,own,NA,NA,2424,24,radio/TV\r\n25,male,2,own,NA,little,8072,30,business\r\n44,female,3,free,little,m

In [7]:
#ruta = '/content/drive/MyDrive/Ricardo Palma/Cursos Tercer Ciclo/Aprendizaje Maquina/Sesión 2/data/'
#ruta = "/content/drive/MyDrive/Cursos/Cursos Tercer Ciclo/Aprendizaje Maquina/Sesión 2/data/"
#train = pd.read_csv(ruta+'german_credit_data_train.csv')
#test = pd.read_csv(ruta+'german_credit_data_train.csv')
train=pd.read_csv('german_credit_data_train.csv')
test=pd.read_csv('german_credit_data_test.csv')


In [8]:
for i in train.columns:
    train[i][train[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=np.nan

In [9]:
for i in test.columns:
    test[i][test[i].apply(lambda i: True if re.search('^\s*$', str(i)) else False)]=np.nan

In [10]:
train.head(10)

,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,NaN,free,NaN,NaN,9055,36,education,good
6,53,male,2,own,quite rich,NaN,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,NaN,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [11]:
train.tail(6)

,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
994,50,male,2,own,NaN,NaN,2390,12,car,good
995,31,female,1,own,little,NaN,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,NaN,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad
999,27,male,2,own,moderate,moderate,4576,45,car,good


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Age               1000 non-null   int64 
 1   Sex               1000 non-null   object
 2   Job               999 non-null    object
 3   Housing           1000 non-null   object
 4   Saving_accounts   817 non-null    object
 5   Checking_account  606 non-null    object
 6   Credit_amount     1000 non-null   int64 
 7   Duration          1000 non-null   int64 
 8   Purpose           1000 non-null   object
 9   Risk              1000 non-null   object
dtypes: int64(3), object(7)
memory usage: 78.2+ KB


In [13]:
#conteo de nulos
missing_tabla(train)

Cantidad de Missing por variable: 


,Cantidad de Missing,% Porcentaje de Missing
Checking_account,394,39.4
Saving_accounts,183,18.3
Job,1,0.1
Age,0,0.0
Sex,0,0.0
Housing,0,0.0
Credit_amount,0,0.0
Duration,0,0.0
Purpose,0,0.0
Risk,0,0.0


In [14]:
train.shape

(1000, 10)

## 3.Tratamiento de la data


In [15]:
# Seleccionamos las variales numericas
feature_num = ['Age','Credit_amount','Duration']
# Al seleccionar el complemento de las
# variables numericas, obtenemos las
# variables categoricas
feature_cat = train.columns.drop(feature_num)
print("Variables numéricas")
print(feature_num)
print("\nVariables categóricas")
print(feature_cat)

Variables numéricas
['Age', 'Credit_amount', 'Duration']

Variables categóricas
Index(['Sex', 'Job', 'Housing', 'Saving_accounts', 'Checking_account',
       'Purpose', 'Risk'],
      dtype='object')


In [16]:
# Observamos las variables numericas
train[feature_num].describe().transpose() 

,count,mean,std,min,25%,50%,75%,max
Age,1000.0,35.546,11.375469,19.0,27.0,33.0,42.00,75.0
Credit_amount,1000.0,3271.258,2822.736876,250.0,1365.5,2319.5,3972.25,18424.0
Duration,1000.0,20.903,12.058814,4.0,12.0,18.0,24.00,72.0


In [17]:
# Observamos las variables categoricas 
for feature in feature_cat:
    print('....................................................................')
    print(train.astype(str).groupby(feature,axis=0)[feature].count().sort_values(ascending=False))

....................................................................
Sex
male      690
female    310
Name: Sex, dtype: int64
....................................................................
Job
2      630
1      199
3      148
0       22
nan      1
Name: Job, dtype: int64
....................................................................
Housing
own     713
rent    179
free    108
Name: Housing, dtype: int64
....................................................................
Saving_accounts
little        603
nan           183
moderate      103
quite rich     63
rich           48
Name: Saving_accounts, dtype: int64
....................................................................
Checking_account
nan         394
little      274
moderate    269
rich         63
Name: Checking_account, dtype: int64
....................................................................
Purpose
car                    337
radio/TV               280
furniture/equipment    181
business                97



## 4.Primer Análisis


Generamos una copia de la data original

In [18]:
data_train = train.copy()

Imputar en primer lugar la data

In [19]:
# moda
#data_train['Job'].fillna('2',inplace=True)
data_train['Job'] = data_train['Job'].fillna('2')
# moda
data_train['Saving_accounts'].fillna('little',inplace=True)
# moda
data_train['Checking_account'].fillna('little',inplace=True)

In [20]:
data_train.head(n=10)

,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
0,67,male,2,own,little,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,little,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
5,35,male,2,free,little,little,9055,36,education,good
6,53,male,2,own,quite rich,little,2835,24,furniture/equipment,good
7,35,male,3,rent,little,moderate,6948,36,car,good
8,61,male,1,own,rich,little,3059,12,radio/TV,good
9,28,male,3,own,little,moderate,5234,30,car,bad


In [21]:
# Dar un orden 
data_train = pd.concat([data_train[feature_num],data_train[feature_cat]],axis=1)

In [22]:
# Columnas: Age, Credit_amount, Duration + categoricas
data_train.head(n=10)

,Age,Credit_amount,Duration,Sex,Job,Housing,Saving_accounts,Checking_account,Purpose,Risk
0,67,1169,6,male,2,own,little,little,radio/TV,good
1,22,5951,48,female,2,own,little,moderate,radio/TV,bad
2,49,2096,12,male,1,own,little,little,education,good
3,45,7882,42,male,2,free,little,little,furniture/equipment,good
4,53,4870,24,male,2,free,little,little,car,bad
5,35,9055,36,male,2,free,little,little,education,good
6,53,2835,24,male,2,own,quite rich,little,furniture/equipment,good
7,35,6948,36,male,3,rent,little,moderate,car,good
8,61,3059,12,male,1,own,rich,little,radio/TV,good
9,28,5234,30,male,3,own,little,moderate,car,bad


Recodificación manual

In [23]:
# Generamos una función por cada variable
dic_Housing={'own':1,'rent':2,'free':3}

def Saving_accounts(n):
   n=str(n)
   if n=='little':
       return 1
   elif  n=='moderate':   
       return 2
   elif  n=='rich':   
       return 3
   else:
       return 4   

def Checking_account(n):
    n=str(n)
    if n=='little':
        return 1
    elif n=='moderate':
        return 2
    else:
        return 3   

def Purpose(n):
   n=str(n)
   if n=='car':
       return 1
   elif  n=='furniture/equipment':   
       return 2
   elif  n=='radio/TV':   
       return 3
   elif  n=='domestic appliances':   
       return 4 
   elif  n=='repairs':   
       return 5 
   elif  n=='education':   
       return 6   
   elif  n=='business':   
       return 7                   
   else:
       return 8   

In [24]:
data_train['Sex']= np.where(data_train['Sex']=='male',2,1) # Operador ternario -> if pero en una sola línea
data_train['Housing'] = data_train['Housing'].map(dic_Housing)
data_train['Saving_accounts']=data_train['Saving_accounts'].apply(Saving_accounts)
data_train['Checking_account']=data_train['Checking_account'].apply(Checking_account)
data_train['Purpose']=data_train['Purpose'].apply(Purpose)
data_train['Risk']=np.where(data_train['Risk']=='bad',1,0)

In [25]:
data_train.dtypes

Age                  int64
Credit_amount        int64
Duration             int64
Sex                  int64
Job                 object
Housing              int64
Saving_accounts      int64
Checking_account     int64
Purpose              int64
Risk                 int64
dtype: object

Ver correlación antes de categorizar

In [29]:
matriz_correlacion, matriz_p_valor = spearmanr(data_train)
df_correlacion = pd.DataFrame(matriz_correlacion, columns=data_train.columns, index=data_train.columns)
df_correlacion

,Age,Credit_amount,Duration,Sex,Job,Housing,Saving_accounts,Checking_account,Purpose,Risk
Age,1.000000,0.026298,-0.036316,0.222706,0.041857,0.003469,-0.004378,-0.054675,-0.033464,-0.112191
Credit_amount,0.026298,1.000000,0.624709,0.115849,0.301337,0.131280,-0.059978,0.000149,-0.074968,0.087083
Duration,-0.036316,0.624709,1.000000,0.073024,0.229688,0.079519,-0.016721,0.002961,0.061913,0.205685
Sex,0.222706,0.115849,0.073024,1.000000,0.062264,-0.085700,-0.001485,-0.013793,0.008985,-0.075493
Job,0.041857,0.301337,0.229688,0.062264,1.000000,0.093364,-0.033823,-0.025766,-0.037601,0.034291
Housing,0.003469,0.131280,0.079519,-0.085700,0.093364,1.000000,-0.011748,-0.009622,-0.101386,0.134305
Saving_accounts,-0.004378,-0.059978,-0.016721,-0.001485,-0.033823,-0.011748,1.000000,0.045902,0.010695,-0.079544
Checking_account,-0.054675,0.000149,0.002961,-0.013793,-0.025766,-0.009622,0.045902,1.000000,0.097335,0.075739
Purpose,-0.033464,-0.074968,0.061913,0.008985,-0.037601,-0.101386,0.010695,0.097335,1.000000,-0.000286
Risk,-0.112191,0.087083,0.205685,-0.075493,0.034291,0.134305,-0.079544,0.075739,-0.000286,1.000000


Colocamos la primera función de correlación

In [30]:
df_correlacion = tidy_corr_matrix(df_correlacion)
df_correlacion['filtro'] = np.where(df_correlacion['abs_r'] > 0.6, 1, 0)

print(df_correlacion)
#df_correlacion.to_csv(ruta+"correlacion_variables.csv", index=False)
df_correlacion.to_csv("correlacion_variables.csv", index=False)

          variable_1        variable_2         r     abs_r  filtro
21          Duration     Credit_amount  0.624709  0.624709       1
12     Credit_amount          Duration  0.624709  0.624709       1
41               Job     Credit_amount  0.301337  0.301337       0
14     Credit_amount               Job  0.301337  0.301337       0
42               Job          Duration  0.229688  0.229688       0
..               ...               ...       ...       ...     ...
36               Sex   Saving_accounts -0.001485  0.001485       0
89           Purpose              Risk -0.000286  0.000286       0
98              Risk           Purpose -0.000286  0.000286       0
71  Checking_account     Credit_amount  0.000149  0.000149       0
17     Credit_amount  Checking_account  0.000149  0.000149       0

[90 rows x 5 columns]


Categorizando a factor

In [31]:
data_train['Job'] = data_train['Job'].astype('int64')

In [32]:
data_train.dtypes

Age                 int64
Credit_amount       int64
Duration            int64
Sex                 int64
Job                 int64
Housing             int64
Saving_accounts     int64
Checking_account    int64
Purpose             int64
Risk                int64
dtype: object

### Primer modelo logístico

In [33]:
# Retiramos la variable correlacionada
data_train = data_train.drop(['Credit_amount'],axis=1)
#X_train['Intercept'] = 1
#y_train = pd.DataFrame(data_train['Risk'],columns=['Risk'],dtype='int64')

In [34]:
data_train

,Age,Duration,Sex,Job,Housing,Saving_accounts,Checking_account,Purpose,Risk
0,67,6,2,2,1,1,1,3,0
1,22,48,1,2,1,1,2,3,1
2,49,12,2,1,1,1,1,6,0
3,45,42,2,2,3,1,1,2,0
4,53,24,2,2,3,1,1,1,1
...,...,...,...,...,...,...,...,...,...
995,31,12,1,1,1,1,1,2,0
996,40,30,2,3,1,1,1,1,0
997,38,12,2,2,1,1,1,3,0
998,23,45,2,2,3,1,1,3,1


In [35]:
# La función sm.add_constant(X)
# agrega una columna con 1 que sirve de intercepto
#X_train_transform = transformar_categoricas(X_train)
#logit_model = sm.Logit(y_train, sm.add_constant(X_train_transform))
logit_model = smf.logit(formula = 'Risk ~ Age + Duration + C(Sex) + C(Job) + C(Housing) + C(Saving_accounts) + C(Checking_account) + C(Purpose)', data = data_train)
#logit_model = smf.logit(formula = 'Risk ~ C(Sex)', data = data_train)

modelo1 = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.559321
         Iterations 6


In [36]:
y_pred = modelo1.predict(sm.add_constant(data_train.drop(['Risk'],axis=1)))
y_pred

0      0.065303
1      0.607831
2      0.224325
3      0.497675
4      0.306126
         ...   
995    0.289563
996    0.305148
997    0.123729
998    0.494297
999    0.551020
Length: 1000, dtype: float64

Indicadores

In [37]:
# Ejecución con variables categoricas tipo de dato int64
modelo1_indicadores = calcula_indicadores(modelo1, data_train.Risk, y_pred)

print(modelo1_indicadores)

      Indicador
GINI   0.392062
AUC    0.696031
KS     0.301429


In [38]:
modelo1.params

Intercept                  -0.950951
C(Sex)[T.2]                -0.329241
C(Job)[T.1]                 0.115286
C(Job)[T.2]                -0.007750
C(Job)[T.3]                 0.033089
C(Housing)[T.2]             0.473069
C(Housing)[T.3]             0.481480
C(Saving_accounts)[T.2]    -0.182651
C(Saving_accounts)[T.3]    -1.159476
C(Saving_accounts)[T.4]    -0.655907
C(Checking_account)[T.2]    0.487220
C(Checking_account)[T.3]   -0.055891
C(Purpose)[T.2]             0.019327
C(Purpose)[T.3]            -0.471409
C(Purpose)[T.4]             0.245812
C(Purpose)[T.5]             0.357025
C(Purpose)[T.6]             0.306459
C(Purpose)[T.7]            -0.066205
C(Purpose)[T.8]            -0.132916
Age                        -0.016723
Duration                    0.036438
dtype: float64

Pesos del modelo importancia mediante la tercera funcion

In [39]:
calcular_pesos(modelo1)

variable
Job                  0.063576
Intercept            3.059981
Sex                  5.027206
Age                  6.265357
Purpose              8.952658
Checking_account    10.632370
Housing             12.110066
Saving_accounts     12.875067
Duration            41.013719
Name: t_value2, dtype: float64

Ver la significancia de modelo

In [40]:
print(modelo1.summary2())

                             Results: Logit
Model:                 Logit              Pseudo R-squared:   0.084     
Dependent Variable:    Risk               AIC:                1160.6416 
Date:                  2021-09-12 13:07   BIC:                1263.7044 
No. Observations:      1000               Log-Likelihood:     -559.32   
Df Model:              20                 LL-Null:            -610.86   
Df Residuals:          979                LLR p-value:        3.5159e-13
Converged:             1.0000             Scale:              1.0000    
No. Iterations:        6.0000                                           
------------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
------------------------------------------------------------------------
Intercept                -0.9510   0.6021 -1.5794 0.1142 -2.1310  0.2291
C(Sex)[T.2]              -0.3292   0.1626 -2.0244 0.0429 -0.6480 -0.0105
C(Job)[

### Segundo modelo logístico

In [41]:
# Reajustando
# Elimina Job
modelo2 = smf.logit(formula = 'Risk ~ Age + Duration + C(Sex) + C(Housing) + C(Saving_accounts) + C(Checking_account) + C(Purpose)', data = data_train).fit()

Optimization terminated successfully.
         Current function value: 0.559523
         Iterations 6


### Calcular indicadores

In [42]:
y_pred = modelo2.predict(sm.add_constant(data_train.drop(['Risk'],axis=1)))
y_pred

0      0.068355
1      0.610889
2      0.210974
3      0.499852
4      0.311646
         ...   
995    0.271022
996    0.302813
997    0.127495
998    0.493818
999    0.557133
Length: 1000, dtype: float64

In [43]:
modelo2_indicadores = calcula_indicadores(modelo2, data_train.Risk, y_pred)

print(modelo2_indicadores)

      Indicador
GINI   0.390357
AUC    0.695179
KS     0.292381


### Pesos del modelo importancia

In [44]:
calcular_pesos(modelo2)

variable
Sex                  4.734642
Age                  5.723494
Intercept            8.312408
Purpose              8.649925
Checking_account    10.310578
Housing             11.233501
Saving_accounts     12.321271
Duration            38.714182
Name: t_value2, dtype: float64